In [1]:
def evaluate(y_test, predictions, heading='-----Evaluation-----'):
    print(heading)
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("\nClassification Report:")
    print(classification_report(y_test, predictions))
    print("Accuracy:", accuracy_score(y_test, predictions))

def remove_files_from_directory(directory):
    # Get all files in the directory
    files = glob.glob(os.path.join(directory, "*"))
    
    # Loop through the files and remove each one
    for file in files:
        if os.path.isfile(file):
            os.remove(file)
    
    print(f"All files in {directory} have been removed.")

def get_anomaly_X_y_from_csv(csv_file, main_labels, target_column, normal_target, output_folder):
    df=pd.read_csv(os.path.join(output_folder, csv_file),usecols=main_labels)
    df=df.fillna(0)
    anomaly_or_not=[]
    for i in df[target_column]: #it changes the normal label to "1" and the anomaly tag to "0" for use in the machine learning algorithm
        if i == normal_target:
            anomaly_or_not.append(1)
        else:
            anomaly_or_not.append(0)           
    df[target_column]=anomaly_or_not

    # y = df[target_column].values
    # del df[target_column]
    # X = df.values
    y_df = df[target_column]
    X_df = df.drop(columns=[target_column])
    
    # X = np.float32(X)
    # X[np.isnan(X)] = 0
    # X[np.isinf(X)] = 0
    # print('X', type(X), X)
    # print('y', type(y), y)
    return (X_df, y_df, df)

def one_hot_encode(df, categorical_columns):
    # Initialize the OneHotEncoder
    ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    
    # Make a copy of the DataFrame to keep track of column order
    X_encoded = df.copy()
    
    # Loop over categorical columns to encode them one by one and retain column order
    for col in categorical_columns:
        # Get the position (index) of the categorical column in the original DataFrame
        col_position = X_encoded.columns.get_loc(col)
        
        # Reshape the column to a 2D array and apply one-hot encoding
        encoded_array = ohe.fit_transform(X_encoded[[col]])
        
        # Create a DataFrame for the one-hot encoded columns
        encoded_columns = ohe.get_feature_names_out([col])
        encoded_df = pd.DataFrame(encoded_array, columns=encoded_columns, index=X_encoded.index)
        
        # Drop the original categorical column from the DataFrame
        X_encoded = X_encoded.drop(columns=[col])
        
        # Insert the one-hot encoded columns back into the DataFrame at the original column's position
        for i, new_col in enumerate(encoded_columns):
            X_encoded.insert(col_position + i, new_col, encoded_df.iloc[:, i])
    
    # X_encoded now contains the one-hot encoded features in the correct positions
    print(list(X_encoded.columns))
    return (ohe, X_encoded)